In [124]:
import json
import os
import unicodedata
import sys
import csv

sys.path.insert(0, '.')

In [25]:
os.chdir('acl_tokenization')

In [26]:
os.getcwd()

'/home/bongseok/Desktop/acl_tokenization'

In [125]:
from tokenizer import (
    # CharTokenizer,
    # JamoTokenizer,
    MeCabSentencePieceTokenizer_orig,
    MeCabSentencePieceTokenizer_fixed,
    MeCabSentencePieceTokenizer,
    MeCabWordPieceTokenizer,
    # MeCabTokenizer,
    MeCabTokenizer_orig,
    MeCabTokenizer_fixed,
    MeCabTokenizer_all,
    # MeCabSentencePieceTokenizer_kortok,
    # MeCabTokenizer_kortok,
    SentencePieceTokenizer,
    WordPieceTokenizer,
    Vocab,
    # WordTokenizer,
)

In [135]:
def get_tokenizer(token_type: str, tokenizer_type: str, decomposition_type: str, space_symbol: str = "", dummy_letter: str = "", grammatical_symbol: list = ["", ""]):
    use_grammatical_symbol = "grammatical_symbol_F" if grammatical_symbol == ["", ""] else "grammatical_symbol_T"

    wp_path = "./resources/v6_without_dummy_letter_" + use_grammatical_symbol
    sub_path = "_".join([token_type, tokenizer_type, decomposition_type, use_grammatical_symbol, "wp-64k"]) + "/bert_tokenizer.json"

    model_path = wp_path + "/" + sub_path
    #print(f"\nmodel path: {model_path}\n")

    wp = WordPieceTokenizer(model_path)


    if (token_type == "eojeol") and (decomposition_type == "composed"):
        tokenizer = wp

    else:
        mecab = MeCabTokenizer_all(token_type=token_type, tokenizer_type=tokenizer_type,
                           decomposition_type=decomposition_type,
                           space_symbol=space_symbol, dummy_letter=dummy_letter,
                           nfd=True, grammatical_symbol=grammatical_symbol)

        tokenizer = MeCabWordPieceTokenizer(mecab=mecab, wp=wp)  # mecab_wp.py

    return tokenizer

In [136]:
def get_tokenized_result(tokenizer, string, nfd: bool = True):
    # if nfd == True:
    #     string = str_to_nfd(string)

    tokenized = tokenizer.tokenize(string)
    print(" ".join(tokenized))

    # nfd 확인용
    # for token in tokenized:
    #     print(f"len: {len(token)}")

In [133]:
def show_tokenizations(string):
    # grammatical symbol F
    tokenizer = get_tokenizer(token_type = "eojeol", tokenizer_type = "mecab_fixed", decomposition_type = "composed", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "eojeol", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_pure", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_orig", decomposition_type = "composed", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_orig", decomposition_type = "decomposed_pure", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "composed", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_pure", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_grammatical", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_lexical", grammatical_symbol = ["", ""])
    get_tokenized_result(tokenizer, string)

    # grammatical symbol T
    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "composed", grammatical_symbol = ["⫸", "⭧"])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_pure", grammatical_symbol = ["⫸", "⭧"])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_grammatical", grammatical_symbol = ["⫸", "⭧"])
    get_tokenized_result(tokenizer, string)

    tokenizer = get_tokenizer(token_type = "morpheme", tokenizer_type = "mecab_fixed", decomposition_type = "decomposed_lexical", grammatical_symbol = ["⫸", "⭧"])
    get_tokenized_result(tokenizer, string)

In [137]:
test1 = '뱌뱌뱌뱌뱌뱌보ㅛ보료르 먹었다.'
test2 = '갸요쟈핿혀뱌ㅐ휴배효ㅗ쀼뤡뙗'
test3 = '철수는 밥을 먹는다.'
test4 = '뱌뵵뵤벼벼벼추퓨를 먹었다.'
print(show_tokenizations(test1), show_tokenizations(test2), show_tokenizations(test3), show_tokenizations(test4))

뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹었다.
ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹었다.
뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹 었 다 .
뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹 ⭧었 ⭧다 .
뱌 ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹 ⭧었 ⭧다 .
뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##뱌 ##보 ##ㅛ ##보 ##료 ##르 먹 ⭧었 ⭧다 .
뱌 ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##ᄇ ##ᅣ ##보 ##ㅛ ##보 ##료 ##르 먹 ⭧었 ⭧다 .
갸 ##요 ##쟈 ##핿 ##혀 ##뱌 ##ㅐ ##휴 ##배 ##효 ##ㅗ ##쀼 ##뤡 ##뙗
갸 ##요 ##쟈 ##해 ##ᆲ ##혀 ##ᄇ ##ᅣ ##ㅐ ##휴 ##배 ##효 ##ㅗ ##ᄈ ##ᅲ ##ᄅ ##ᅰ ##ᆨ ##ᄄ ##ᅫ ##ᆶ
갸 요 쟈
갸 요 쟈 해 ##ᆲ ##혀 ##ᄇ ##ᅣ ##ㅐ ##휴 ##배 ##효 ##ㅗ ##ᄈ ##ᅲ ##ᄅ #

In [138]:
show_tokenizations(test4)

먹었다.
ᄇ ##ᅣ ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹었다.
먹 었 다 .
뱌 ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹 었 다 .
먹 었 다 .
뱌 ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹 었 다 .
먹 었 다 .
뱌 ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹 었 다 .
먹 ⭧었 ⭧다 .
뱌 ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹 ⭧었 ⭧다 .
먹 ⭧었 ⭧다 .
뱌 ##ᄇ ##ᅭ ##ᆸ ##ᄇ ##ᅭ ##벼 ##벼 ##벼 ##추 ##퓨 ##를 먹 ⭧었 ⭧다 .


In [139]:
# 예시 문장 불러오기
# cola test를 한 줄씩 불러와서 sent 변수에 입력
# sent를 토큰화
# 원문장, sent를 csv로 저장

# .tsv 읽기
# import csv
# f = open('dataset/nlu_tasks/korsts/sts-test.tsv', 'r', encoding='utf-8')
# reader = csv.reader(f, delimiter=',')
# lines = list(reader)
# f.close()

# sents = []
# for line in lines:
#     sents.append(line[0])
# print(sents)
# len(sents)


import pandas as pd

data = pd.read_csv('dataset/nlu_tasks/paws/test_2k.tsv', sep = '\t')


In [140]:
data

id                                          sentence1  \
0       10  2005년과 2009년 사이 그가 스웨덴 Carlstad United BK, 세르비...   
1       12                           타바시 강은 루마니아 류드라 강의 지류이다.   
2       20  그는 1993년에는 싱글A Kane County Cougars에서 선수 생활을 했고...   
3       26             위나스키는 IEEE, 피 베타카파, ACM과 시그마 Xi의 회원이다.   
4       27  1938년 그는 앵글로 이집트 수단의 정부 인류학자가 되었고 Nuba와 함께 현장 ...   
...    ...                                                ...   
1995  7984  Bas Basie Land는 카운트 베이시와 그의 오케스트라가 1964년에 제작한 ...   
1996  7992   그들의 임무는 들이닥치는 적 함대에 대항하여 다양한 우주 전초 기지를 지키는 것입니다.   
1997  7993  그의 어머니 Elizabeth 쪽에는 Cornwall 공국 수상인 William M...   
1998  7999  2014년에 이 사이트는 제품 검색을 위한 iOS 및 Android 애플리케이션을 ...   
1999  8000                 다른 방향에서 ``m``은 주 인수분해를 포함하여 강력합니다.   

                                              sentence2  label  
0     2005년 후반에서 2009년 사이 그가 스웨덴 Carlstad United BK,...    1.0  
1                     Leurda강은 루마니아에 있는 Tabaci강의 지류입니다.    0.0  
2     1993년에는 A레벨 Portland Sea Dogs 및 AA Kane County...    0.0  
3              위너스키는 ACM과 IEEE, 피 베타 카파회, 시그마 시회의 회원이다.    1.0  
4     그는 1938년에 앵글로 이집트 수단의 정부 인류학자가 되었으며 Nuba와 함께 현...    0.0  
...                                                 ...    ...  
1995  Land Basie Land는 Billy Byers와 그의 오케스트라의 1964년 ...    0.0  
1996        그들의 임무는 다양한 전초 기지를 수많은 적군의 선박으로부터 지키는 것입니다.    0.0  
1997  그의 어머니 쪽에는 Elizabeth William Mordaunt, Cornwal...    0.0  
1998  2014년, iOS 및 Android 현장에서는 제품 검색 응용 프로그램, 제품 -...    1.0  
1999         다른 방향에서, "m"은 강력한 인수분해와 함께 주요한 것으로 간주하십시오.    0.0  

[2000 rows x 4 columns]

In [141]:
sent1 = data['sentence1'].copy()
sent2 = data['sentence2'].copy()

In [142]:
sent1

0       2005년과 2009년 사이 그가 스웨덴 Carlstad United BK, 세르비...
1                                타바시 강은 루마니아 류드라 강의 지류이다.
2       그는 1993년에는 싱글A Kane County Cougars에서 선수 생활을 했고...
3                  위나스키는 IEEE, 피 베타카파, ACM과 시그마 Xi의 회원이다.
4       1938년 그는 앵글로 이집트 수단의 정부 인류학자가 되었고 Nuba와 함께 현장 ...
                              ...                        
1995    Bas Basie Land는 카운트 베이시와 그의 오케스트라가 1964년에 제작한 ...
1996     그들의 임무는 들이닥치는 적 함대에 대항하여 다양한 우주 전초 기지를 지키는 것입니다.
1997    그의 어머니 Elizabeth 쪽에는 Cornwall 공국 수상인 William M...
1998    2014년에 이 사이트는 제품 검색을 위한 iOS 및 Android 애플리케이션을 ...
1999                   다른 방향에서 ``m``은 주 인수분해를 포함하여 강력합니다.
Name: sentence1, Length: 2000, dtype: object

In [109]:
tokenized = [show_tokenizations(string=sent) for sent in sent1]

# tokenizing_test(sents)

2005년 ##과 2009년 사이 그가 스웨덴 Car ##l ##st ##ad Un ##ited B ##K, 세르비아 FK Bo ##ra ##c 러시아 FC Te ##re ##k Gr ##o ##z ##ny ##에서 뛰었던 것은 제외 ##됩니다.
2005년 ##과 2009년 사이 그가 스웨덴 Car ##l ##st ##ad United B ##K, 세르비아 FK Bo ##ra ##c Ca ##c ##ak ##, 러시아 FC Te ##re ##k Gro ##z ##ny ##에서 뛰었던 것은 제외 ##됩니다.
2005 년 과 2009 년 사이 그 가 스웨덴 Carl ##st ##ad United BK , 세르비아 FK Bor ##ac , 러시아 FC Ter ##e ##k Gro ##z ##ny 에서 뛰 었 던 것 은 제외 됩니다 .
2005 년 과 2009 년 사이 그 가 스웨덴 Carl ##st ##ad United BK , 세르비아 FK Bor ##ac Ca ##ca ##k , 러시아 FC Ter ##e ##k Gro ##z ##ny 에서 뛰 었 던 것 은 제외 됩니다 .
2005 년 과 2009 년 사이 그 가 스웨덴 Carl ##st ##ad United BK , 세르비아 FK Bor ##ac , 러시아 FC Ter ##e ##k Gro ##z ##ny 에서 뛰 었 던 것 은 제외 되 ᄇ니다 .
2005 년 과 2009 년 사이 그 가 스웨덴 Carl ##st ##ad United BK , 세르비아 FK Bor ##ac Ca ##ca ##k , 러시아 FC Ter ##ek Gro ##z ##ny 에서 뛰 었 던 것 은 제외 되 ᄇ니다 .
2005 년 과 2009 년 사이 그 가 스웨덴 Carl ##st ##ad United BK , 세르비아 FK Bor ##a

이 요구사항 ⫸에 대하 ⭧ᆫ 태양열 접근 방식 ⫸은 재래식 추진 항공기 ⫸에 태양열 전 지판 ⫸을 사용 하 ⭧는 것 이 ⭧ᄇ니다 .
이 요구사항 ⫸에 대하 ⭧ᆫ 태양열 접근 방식 ⫸은 재래식 추진 항공기 ⫸에 태양열 전 지판 ⫸을 사용 하 ⭧는 것 이 ⭧ᄇ니다 .
이 요구사항 ⫸에 대하 ⭧ᆫ 태양열 접근 방식 ⫸은 재래식 추진 항공기 ⫸에 태양열 전 지판 ⫸을 사용 하 ⭧는 것 이 ⭧ᄇ니다 .
경찰은 S ##id ##d ##i ##que ##와 그의 부인 Di ##leep ##의 절친 ##인 가수 R ##im ##i Tom ##y ##와 배우 K ##av ##ya Mad ##ha ##va ##n ##에게도 수사 ##의 일환으로 심문 ##했습니다.
경찰은 S ##id ##d ##ique ##와 그의 부인 Di ##leep ##의 절친 ##인 가수 R ##im ##i Tom ##y ##와 배우 Ka ##vy ##a Mad ##ha ##va ##n ##에게도 수사 ##의 일환으로 심문 ##했습니다.
경찰 은 Si ##dd ##ique 와 그 의 부인 Di ##leep 의 절친 인 가수 R ##im ##i Tom ##y 와 배우 Ka ##vy ##a Mad ##ha ##va ##n 에게 도 수사 의 일환 으로 심문 했 습니다 .
경찰 은 Si ##dd ##ique 와 그 의 부인 Di ##leep 의 절친 인 가수 Ri ##mi Tom ##y 와 배우 Ka ##vy ##a Mad ##ha ##van 에게 도 수사 의 일환 으로 심문 했 습니다 .
경찰 은 Si ##dd ##ique 와 그 의 부

2009 년 에 그 는 필라델피아 로 돌아오 았 으며 지금 은 뉴욕 시 에 살 고 있 습니다 .
2009 년 에 그 는 필라델피아 로 돌아오 았 으며 지금 은 뉴욕 시 에 살 고 있 습니다 .
2009 년 에 그 는 필라델피아 로 돌아오 았 으며 지금 은 뉴욕 시 에 살 고 있 습니다 .
2009 년 에 그 는 필라델피아 로 돌아오 았 으며 지금 은 뉴욕 시 에 살 고 있 습니다 .
2009 년 ⫸에 그 ⫸는 필라델피아 ⫸로 돌아오 ⭧았 ⭧으며 지금 ⫸은 뉴욕 시 ⫸에 살 ⭧고 있 ⭧습니다 .
2009 년 ⫸에 그 ⫸는 필라델피아 ⫸로 돌아오 ⭧았 ⭧으며 지금 ⫸은 뉴욕 시 ⫸에 살 ⭧고 있 ⭧습니다 .
2009 년 ⫸에 그 ⫸는 필라델피아 ⫸로 돌아오 ⭧았 ⭧으며 지금 ⫸은 뉴욕 시 ⫸에 살 ⭧고 있 ⭧습니다 .
2009 년 ⫸에 그 ⫸는 필라델피아 ⫸로 돌아오 ⭧았 ⭧으며 지금 ⫸은 뉴욕 시 ⫸에 살 ⭧고 있 ⭧습니다 .
그는 1954년 6월 30일 L ##yn ##n R ##ig ##g ##s Mem ##ori ##al ##이 있는 뉴욕 ##시 오클라호마 ##주 클레어 ##모어 ##에서 위 ##암으로 사망 ##했습니다.
그는 1954년 6월 30일 L ##yn ##n R ##ig ##g ##s Mem ##ori ##al ##이 있는 뉴욕 ##시 오클라호마 ##주 클레어 ##모어 ##에서 위 ##암으로 사망 ##했습니다.
그 는 1954 년 6 월 30 일 Ly ##n ##n R ##ig ##g ##s Mem ##oria ##l 이 있 는 뉴욕 시 오클라호마 ##주 클레어 ##모어 에서

영화 는 상업 적 으로 성공 하 았 고 Ser ##gi ##o Sol ##lim ##a 의 성공 적 이 ᆫ 영화 중 하나 이 었 으며 예전 스파게티 웨스턴 감독 이 하 았 던 작품 들 보다 정치 ##색 이 덜 하 았 습니다 .
영화 ⫸는 상업 적 ⫸으로 성공 하 ⭧았 ⭧고 Ser ##gi ##o Sol ##lim ##a ⫸의 성공 적 이 ⭧ᆫ 영화 중 하나 이 ⭧었 ⭧으며 예전 스파게티 웨스턴 감독 ⫸이 하 ⭧았 ⭧던 작품 들 ⫸보다 정치 ##색 ⫸이 덜 하 ⭧았 ⭧습니다 .
영화 ⫸는 상업 적 ⫸으로 성공 하 ⭧았 ⭧고 Ser ##gi ##o Sol ##lim ##a ⫸의 성공 적 이 ⭧ᆫ 영화 중 하나 이 ⭧었 ⭧으며 예전 스파게티 웨스턴 감독 ⫸이 하 ⭧았 ⭧던 작품 들 ⫸보다 정치 ##색 ⫸이 덜 하 ⭧았 ⭧습니다 .
영화 ⫸는 상업 적 ⫸으로 성공 하 ⭧았 ⭧고 Ser ##gi ##o Sol ##lim ##a ⫸의 성공 적 이 ⭧ᆫ 영화 중 하나 이 ⭧었 ⭧으며 예전 스파게티 웨스턴 감독 ⫸이 하 ⭧았 ⭧던 작품 들 ⫸보다 정치 ##색 ⫸이 덜 하 ⭧았 ⭧습니다 .
영화 ⫸는 상업 적 ⫸으로 성공 하 ⭧았 ⭧고 Ser ##gi ##o Sol ##lim ##a ⫸의 성공 적 이 ⭧ᆫ 영화 중 하나 이 ⭧었 ⭧으며 예전 스파게티 웨스턴 감독 ⫸이 하 ⭧았 ⭧던 작품 들 ⫸보다 정치 ##색 ⫸이 덜 하 ⭧았 ⭧습니다 .
K ##ab ##ir ##는 S ##ari ##ka ##에게 R ##an ##v ##ir ##의 게임을 끝낼 계획을 알려 ##달라고 부탁

영국 에서 는 일반 적 이 었 지만 유럽 에서 는 적어도 대형 기관차 의 경우 상대 적 으로 드 무 ##ᆸ니다 .
영국 에서 는 일반 적 이 었 지만 유럽 에서 는 적어도 대형 기관차 의 경우 상대 적 으로 드 뭅 ##니다 .
영국 에서 는 일반 적 이 었 지만 유럽 에서 는 적어도 대형 기관차 의 경우 상대 적 으로 드 무 ##ᆸ니다 .
영국 에서 는 일반 적 이 었 지만 유럽 에서 는 적어도 대형 기관차 의 경우 상대 적 으로 드 뭅 ##니다 .
영국 에서 는 일반 적 이 었 지만 유럽 에서 는 적어도 대형 기관차 의 경우 상대 적 으로 드 무 ##ᆸ니다 .
영국 ⫸에서 ⫸는 일반 적 이 ⭧었 ⭧지만 유럽 ⫸에서 ⫸는 적어도 대형 기관차 ⫸의 경우 상대 적 ⫸으로 드 뭅 ##니다 .
영국 ⫸에서 ⫸는 일반 적 이 ⭧었 ⭧지만 유럽 ⫸에서 ⫸는 적어도 대형 기관차 ⫸의 경우 상대 적 ⫸으로 드 무 ##ᆸ니다 .
영국 ⫸에서 ⫸는 일반 적 이 ⭧었 ⭧지만 유럽 ⫸에서 ⫸는 적어도 대형 기관차 ⫸의 경우 상대 적 ⫸으로 드 뭅 ##니다 .
영국 ⫸에서 ⫸는 일반 적 이 ⭧었 ⭧지만 유럽 ⫸에서 ⫸는 적어도 대형 기관차 ⫸의 경우 상대 적 ⫸으로 드 무 ##ᆸ니다 .
Al ##st ##on ##은 1965년 12월 21일 메릴 ##랜드 ##주 O ##x ##on Hi ##ll ##에서 태어났 ##습니다. 그는 코네 ##티 ##컷 ##주 New Ha ##ven ##에 있는 O ##x ##on Hi #

...

Bob Cour ##ts ⫸의 " Black ##add ##er " ⫸로 연주 하 ⭧ᆫ 음악 ⫸은 Va ##ugh ##an William ##s ⫸의 " Fant ##asia on Green ##s ##lee ##ves " 이 ⭧ᄇ니다 .
Bob Court ##s ⫸의 " Black ##add ##er " ⫸로 연주 하 ⭧ᆫ 음악 ⫸은 Va ##ugh ##an William ##s ⫸의 " Fant ##asia on Green ##s ##lee ##ves " 이 ⭧ᄇ니다 .
Bob Cour ##ts ⫸의 " Black ##add ##er " ⫸로 연주 하 ⭧ᆫ 음악 ⫸은 Va ##ugh ##an William ##s ⫸의 " Fant ##asia on Green ##s ##lee ##ves " 이 ⭧ᄇ니다 .
Bob Court ##s ⫸의 " Black ##add ##er " ⫸로 연주 하 ⭧ᆫ 음악 ⫸은 Va ##ugh ##an William ##s ⫸의 " Fant ##asia on Green ##s ##lee ##ves " 이 ⭧ᄇ니다 .
다음 코드는 AD ##L 없이 작동 ##이 가능하다는 것을 알아 ##두는 것도 좋 ##습니다. ( ##상 ##관 없이 적용될 것입니다. ##)
다음 코드는 AD ##L 없이 작동 ##이 가능하다는 것을 알아 ##두는 것도 좋 ##습니다. ( ##상과 ##ᆫ 없이 적용될 것입니다. ##)
다음 코드 는 AD ##L 없이 작동 이 가능 하 다는 것 을 알 아 두 는 것 도 좋 습니다 . ( 상관 없이 적용 될 것 입니다 . )
다음 코드 는 AD ##L 없이 작동 이 가능 하 다는 것 을 알 아 두 는 것 도 좋 습니다 . ( 상관 없이 적용 될 것 입니다 

Kar ##en Phil ##li ##ps ⫸는 Sur ##ann ##e Jon ##es ⫸가 연기 하 ⭧ᆫ Steve ⫸와 만나 ⭧기 시작 하 ⭧았 ⭧습니다 .
Kar ##en Phil ##li ##ps ⫸는 Sur ##anne Jones ⫸가 연기 하 ⭧ᆫ Steve ⫸와 만나 ⭧기 시작 하 ⭧았 ⭧습니다 .
Kar ##en Phil ##li ##ps ⫸는 Sur ##ann ##e Jon ##es ⫸가 연기 하 ⭧ᆫ Ste ##ve ⫸와 만나 ⭧기 시작 하 ⭧았 ⭧습니다 .
Kar ##en Phil ##li ##ps ⫸는 Sur ##anne Jones ⫸가 연기 하 ⭧ᆫ Steve ⫸와 만나 ⭧기 시작 하 ⭧았 ⭧습니다 .
비 ##다스 디스 ##틴 ##타스 ##는 멕시코의 TV ##연속 ##극 ##으로 1969년 Te ##les ##ist ##em ##a Me ##x ##ican ##o ##가 제작하여 Te ##le ##v ##is ##a가 방영 ##하였다.
비 ##다스 디스 ##틴 ##타스 ##는 멕시코의 TV ##연속 ##극으로 1969년 Te ##les ##ist ##em ##a Me ##x ##ican ##o가 제작하여 Te ##le ##vi ##s ##a가 방영 ##하였다.
비 다스 디스 틴 타스 는 멕시코 의 TV 연속극 으로 1969 년 Tele ##s ##ist ##em ##a Me ##x ##ican ##o 가 제작 하 여 Tele ##vis ##a 가 방영 하 였 다 .
비 다스 디스 틴 타스 는 멕시코 의 TV 연속극 으로 1969 년 Tele ##si ##stem ##a Me ##x ##ican ##o 가 제작 하 여 Tele ##vis ##a 가 방영 하 였 다 .
비 다스 디스 틴 타스 는 멕시코 의 TV 연속

파리 ⫸에서 10 년 ⫸을 지내 ⭧ᆫ 후 Ho ##udo ##n ⫸은 이태리 ⫸로 돌아오 ⭧았 ⭧습니다 .
이는 독일에 의해 침몰 ##하여 194 ##3 ##-19 ##44 ##년 연합군 폭격기 ##에 의해 두 조각 ##으로 부서 ##졌고, 194 ##6 ##-19 ##47 ##년에 인 ##양 ##되었다.
이는 독일에 의해 침몰 ##하여 194 ##3 ##-19 ##44 ##년 연합군 폭격기 ##에 의해 두 조각으로 부서져 ##ᆻ고, 194 ##6 ##-19 ##47 ##년에 인양 ##되었다.
이 는 독일 에 의해 침몰 하 여 1943 - 1944 년 연합군 폭격기 에 의해 두 조각 으로 부서졌 고 , 1946 - 1947 년 에 인양 되 었 다 .
이 는 독일 에 의해 침몰 하 여 1943 - 1944 년 연합군 폭격기 에 의해 두 조각 으로 부서졌 고 , 1946 - 1947 년 에 인양 되 었 다 .
이 는 독일 에 의하 아 침몰 하 여 1943 - 1944 년 연합군 폭격기 에 의하 아 두 조각 으로 부서지 었 고 , 1946 - 1947 년 에 인양 되 었 다 .
이 는 독일 에 의하 아 침몰 하 여 1943 - 1944 년 연합군 폭격기 에 의하 아 두 조각 으로 부서지 었 고 , 1946 - 1947 년 에 인양 되 었 다 .
이 는 독일 에 의하 아 침몰 하 여 1943 - 1944 년 연합군 폭격기 에 의하 아 두 조각 으로 부서지 었 고 , 1946 - 1947 년 에 인양 되 었 다 .
이 는 독일 에 의하 아 침몰 하 여 1943 - 1944 년 연합군 폭겨

프로 ##보 키 눔 터 이 네 룸 은 바다 우렁 이 종 으로 , 바다 고둥 이 라고 도 하 는 물레 ##고 ##둥 과 복 ##족 류 연체 ##동물 이 다 .
프로 ##보 키 눔 터 이 네 룸 은 바다 우렁 이 종 으로 , 바다 고둥 이 라고 도 하 는 물레 ##고 ##둥 과 복 ##족 류 연체 ##동물 이 다 .
프로 ##보 키 눔 터 이 네 룸 은 바다 우렁 이 종 으로 , 바다 고둥 이 라고 도 하 는 물레 ##고 ##둥 과 복 ##족 류 연체 ##동물 이 다 .
프로 ##보 키 눔 터 이 ⭧네 룸 ⫸은 바다 우렁 ⫸이 종 ⫸으로 , 바다 고둥 이 ⭧라고 ⫸도 하 ⭧는 물레 ##고 ##둥 ⫸과 복 ##족 류 연체 ##동물 이 ⭧다 .
프로 ##보 키 눔 터 이 ⭧네 룸 ⫸은 바다 우렁 ⫸이 종 ⫸으로 , 바다 고둥 이 ⭧라고 ⫸도 하 ⭧는 물레 ##고 ##둥 ⫸과 복 ##족 류 연체 ##동물 이 ⭧다 .
프로 ##보 키 눔 터 이 ⭧네 룸 ⫸은 바다 우렁 ⫸이 종 ⫸으로 , 바다 고둥 이 ⭧라고 ⫸도 하 ⭧는 물레 ##고 ##둥 ⫸과 복 ##족 류 연체 ##동물 이 ⭧다 .
프로 ##보 키 눔 터 이 ⭧네 룸 ⫸은 바다 우렁 ⫸이 종 ⫸으로 , 바다 고둥 이 ⭧라고 ⫸도 하 ⭧는 물레 ##고 ##둥 ⫸과 복 ##족 류 연체 ##동물 이 ⭧다 .
Val ##ea l ##u ##i 강 또는 Lo ##m 강은 루마니아 ##에 있는 강의 지 ##류 ##입니다.
Val ##ea l ##u ##i La ##mb ##a 강 또는 Lo ##m 강은 루마니ᄋ

KeyboardInterrupt: 

True